In [7]:
%pip install --upgrade numpy yfinance pandas matplotlib xgboost nltk scipy scikit-learn tensorflow joblib h5py

Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow-2.20.0-cp313-cp313-win_amd64.whl.metadata (4.6 kB)
Using cached tensorflow-2.20.0-cp313-cp313-win_amd64.whl (332.0 MB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\prana\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python313\\site-packages\\tensorflow\\include\\external\\com_github_grpc_grpc\\src\\core\\ext\\filters\\fault_injection\\fault_injection_service_config_parser.h'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\prana\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import os
import requests
import time
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import yfinance as yf
from pathlib import Path
import numpy as np
import joblib
import warnings
warnings.filterwarnings('ignore')

# Scikit-learn imports
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, mean_squared_error, mean_absolute_error, r2_score
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import cdist
from sklearn.metrics import accuracy_score
import xgboost as xgb

# TensorFlow/Keras imports for LSTM
_tf_available = False
try:
    from tensorflow import keras
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense, Dropout
    from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
    _tf_available = True
    print("✓ TensorFlow/Keras imported successfully")
except ImportError as e1:
    try:
        from keras.models import Sequential
        from keras.layers import LSTM, Dense, Dropout
        from keras.callbacks import EarlyStopping, ModelCheckpoint
        _tf_available = True
        print("✓ Keras imported successfully")
    except ImportError as e2:
        print("\n" + "="*70)
        print("❌ ERROR: TensorFlow/Keras is not installed or not importable!")
        print("="*70)
        print("To fix this issue:")
        print("1. Install TensorFlow: %pip install tensorflow")
        print("2. If you get Windows Long Path errors, enable Long Path support:")
        print("   - Run PowerShell as Administrator")
        print("   - Execute: New-ItemProperty -Path 'HKLM:\SYSTEM\CurrentControlSet\Control\FileSystem' -Name 'LongPathsEnabled' -Value 1 -PropertyType DWORD -Force")
        print("   - Restart your computer")
        print("3. Alternative: Try installing with: %pip install tensorflow-cpu")
        print("="*70)
        raise ImportError("TensorFlow/Keras is required for LSTM models but could not be imported. "
                         f"TensorFlow error: {e1}, Standalone Keras error: {e2}")

⚠ Warning: TensorFlow/Keras not found. Please install: pip install tensorflow


In [9]:
TOP_50_TICKERS = [
    "NVDA", "AAPL", "GOOG", "MSFT", "AMZN", "META", "TSLA", "AVGO", "2222.SR", "TSM",
    "BRK-B", "LLY", "WMT", "JPM", "TCEHY", "V", "ORCL", "MA", "005930.KS", "XOM",
    "JNJ", "PLTR", "BAC", "ASML", "ABBV", "NFLX", "601288.SS", "COST", "MC.PA", "BABA",
    "1398.HK", "AMD", "HD", "601939.SS", "ROG.SW", "PG", "GE", "MU", "CSCO", "KO",
    "WFC", "CVX", "UNH", "MS", "SAP", "TM", "AZN", "IBM", "CAT", "000660.KS"
]

current_dir = Path.cwd()
project_root = current_dir if current_dir.name != 'Notebooks' else current_dir.parent
raw_data_path = project_root / "Data" 
raw_data_path.mkdir(parents=True, exist_ok=True)

def fetch_tickers_in_batches(tickers, batch_size=10, period="6mo"):
    all_data = []
    for i in range(0, len(tickers), batch_size):
        batch = tickers[i:i + batch_size]
        print(f"Downloading batch: {batch}")
        try:
            data = yf.download(batch, period=period, group_by='ticker', auto_adjust=True, threads=True)
            if not data.empty:
                all_data.append(data)
        except Exception as e:
            print(f"Batch download error: {e}")
        time.sleep(1)
    return pd.concat(all_data, axis=1) if all_data else pd.DataFrame()

def robust_downloader(tickers, period="6mo", max_retries=3):
    print(f"Initiating resilient download for {len(tickers)} tickers...")
    df = fetch_tickers_in_batches(tickers, batch_size=15, period=period)
    
    for attempt in range(max_retries):
        existing_tickers = df.columns.get_level_values(0).unique().tolist()
        failed_tickers = [t for t in tickers if t not in existing_tickers]
        
        if not failed_tickers:
            print("All tickers downloaded successfully.")
            break
            
        wait_time = (attempt + 1) * 5
        print(f"Attempt {attempt + 1}/{max_retries}: {len(failed_tickers)} failures. Retrying in {wait_time}s...")
        time.sleep(wait_time)
        
        for ticker in failed_tickers:
            try:
                retry_data = yf.download(ticker, period=period, auto_adjust=True, progress=False)
                if not retry_data.empty and len(retry_data) > 20:
                    # Fix MultiIndex issue
                    if not isinstance(retry_data.columns, pd.MultiIndex):
                        retry_data.columns = pd.MultiIndex.from_product([[ticker], retry_data.columns])
                    df = pd.concat([df, retry_data], axis=1)
                    print(f"✓ Successfully retrieved {ticker}")
            except Exception as e:
                print(f"✗ Failed for {ticker}: {str(e)[:50]}")

    df = df.sort_index().interpolate(method='time').ffill().bfill()
    
    missing_final = [t for t in tickers if t not in df.columns.get_level_values(0).unique()]
    if missing_final:
        print(f"⚠ CRITICAL: Still missing {len(missing_final)} tickers: {missing_final}")
    
    file_path = raw_data_path / "market_data_raw.csv"
    df.to_csv(file_path)
    print(f"✓ Data ingestion complete. File saved: {file_path}")
    print(f"✓ Successfully collected {df.columns.get_level_values(0).nunique()} tickers")
    return df

raw_df = robust_downloader(TOP_50_TICKERS)

Initiating resilient download for 50 tickers...


[*********************100%***********************]  15 of 15 completed
[*************         27%                       ]  4 of 15 completed

[*********************100%***********************]  15 of 15 completed
[******                13%                       ]  2 of 15 completed

[*********************100%***********************]  15 of 15 completed


[*********************100%***********************]  5 of 5 completed


All tickers downloaded successfully.
✓ Data ingestion complete. File saved: d:\VSCode\Projects\Stock-Market-Trend-Analysis\Data\market_data_raw.csv
✓ Successfully collected 50 tickers


In [10]:
# Add this cell to check your data:
df = pd.read_csv(proc_dir / "market_data_clusters.csv")
print(f"Total rows: {len(df)}")
print(f"Tickers: {df['Ticker'].nunique()}")
print(f"Date range: {df['Date'].min()} to {df['Date'].max()}")
print(f"\nRows per ticker:")
print(df['Ticker'].value_counts().describe())
print(f"\nMissing values:")
print(df.isnull().sum())

Total rows: 6950
Tickers: 50
Date range: 2025-08-04 to 2026-01-12

Rows per ticker:
count     50.0
mean     139.0
std        0.0
min      139.0
25%      139.0
50%      139.0
75%      139.0
max      139.0
Name: count, dtype: float64

Missing values:
Date              0
Open              0
High              0
Low               0
Close             0
Volume            0
RSI_14            0
SMA_5             0
SMA_20            0
SMA_Ratio_5_20    0
MACD              0
Signal            0
MACD_Histogram    0
BB_Upper          0
BB_Lower          0
BB_Position       0
Vol_SMA_20        0
Volume_Ratio      0
Ticker            0
Anomaly_Flag      0
Is_Anomaly        0
Cluster           0
dtype: int64


In [11]:
current_dir = Path.cwd()
project_root = current_dir if current_dir.name != 'Notebooks' else current_dir.parent
raw_file = project_root / "Data" / "market_data_raw.csv"
proc_dir = project_root / "Data" 
proc_dir.mkdir(parents=True, exist_ok=True)

def calculate_indicators(df):
    """Calculates technical indicators"""
    df = df.copy()  # Avoid SettingWithCopyWarning
    
    # RSI_14
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['RSI_14'] = 100 - (100 / (1 + rs))

    # SMA Ratio 5/20
    df['SMA_5'] = df['Close'].rolling(window=5).mean()
    df['SMA_20'] = df['Close'].rolling(window=20).mean()
    df['SMA_Ratio_5_20'] = df['SMA_5'] / df['SMA_20']

    # MACD Histogram
    ema12 = df['Close'].ewm(span=12, adjust=False).mean()
    ema26 = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = ema12 - ema26
    df['Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    df['MACD_Histogram'] = df['MACD'] - df['Signal']

    # Bollinger Bands Position
    std = df['Close'].rolling(window=20).std()
    df['BB_Upper'] = df['SMA_20'] + (std * 2)
    df['BB_Lower'] = df['SMA_20'] - (std * 2)
    df['BB_Position'] = (df['Close'] - df['BB_Lower']) / (df['BB_Upper'] - df['BB_Lower'])

    # Volume Ratio
    df['Vol_SMA_20'] = df['Volume'].rolling(window=20).mean()
    df['Volume_Ratio'] = df['Volume'] / df['Vol_SMA_20']

    return df.dropna()

print("Loading raw data and calculating features...")
df_raw = pd.read_csv(raw_file, header=[0, 1], index_col=0, parse_dates=True)
tickers = df_raw.columns.get_level_values(0).unique()

all_processed = []
for ticker in tickers:
    try:
        stock_data = df_raw[ticker].copy()
        if stock_data.empty or len(stock_data) < 50:
            print(f"⚠ Skipping {ticker}: Insufficient data ({len(stock_data)} days)")
            continue 

        processed_stock = calculate_indicators(stock_data)
        processed_stock['Ticker'] = ticker  # No more SettingWithCopyWarning
        all_processed.append(processed_stock)
        print(f"✓ Processed {ticker}: {len(processed_stock)} days")
    except Exception as e:
        print(f"✗ Error processing {ticker}: {str(e)[:50]}")

final_df = pd.concat(all_processed)
final_df.to_csv(proc_dir / "market_data_features.csv")
print(f"\n✓ Total: {len(final_df)} rows across {final_df['Ticker'].nunique()} tickers")
print(f"✓ File saved: {proc_dir / 'market_data_features.csv'}")

Loading raw data and calculating features...
✓ Processed META: 140 days
✓ Processed GOOG: 140 days
✓ Processed AVGO: 140 days
✓ Processed TSLA: 140 days
✓ Processed BRK-B: 140 days
✓ Processed TCEHY: 140 days
✓ Processed AMZN: 140 days
✓ Processed 2222.SR: 140 days
✓ Processed NVDA: 140 days
✓ Processed JPM: 140 days
✓ Processed TSM: 140 days
✓ Processed LLY: 140 days
✓ Processed AAPL: 140 days
✓ Processed MSFT: 140 days
✓ Processed WMT: 140 days
✓ Processed BABA: 140 days
✓ Processed PLTR: 140 days
✓ Processed NFLX: 140 days
✓ Processed 601288.SS: 140 days
✓ Processed ORCL: 140 days
✓ Processed ASML: 140 days
✓ Processed COST: 140 days
✓ Processed ABBV: 140 days
✓ Processed V: 140 days
✓ Processed 005930.KS: 140 days
✓ Processed MC.PA: 140 days
✓ Processed MA: 140 days
✓ Processed XOM: 140 days
✓ Processed BAC: 140 days
✓ Processed JNJ: 140 days
✓ Processed MS: 140 days
✓ Processed AMD: 140 days
✓ Processed SAP: 140 days
✓ Processed UNH: 140 days
✓ Processed WFC: 140 days
✓ Processed 

In [12]:
anomaly_df = pd.read_csv(proc_dir / "market_data_features.csv")
model_root = project_root / "Models"
model_root.mkdir(parents=True, exist_ok=True)

features = [
    'RSI_14',
    'SMA_Ratio_5_20',
    'MACD_Histogram',
    'BB_Position',
    'Volume_Ratio'
]

clean_anomaly_df = anomaly_df.dropna(subset=features).copy()

iso_model = IsolationForest(n_estimators=100, max_samples='auto', contamination=0.05, random_state=42, max_features=len(features), n_jobs=-1)

clean_anomaly_df['Anomaly_Flag'] = iso_model.fit_predict(clean_anomaly_df[features])
clean_anomaly_df['Is_Anomaly'] = clean_anomaly_df['Anomaly_Flag'].apply(lambda x: 1 if x == -1 else 0)

joblib.dump(iso_model, model_root / "isolation_forest_model.pkl")
clean_anomaly_df.to_csv(proc_dir / "market_data_anomalies.csv", index=False)

print(f"Detected {clean_anomaly_df['Is_Anomaly'].sum()} anomalies across all tickers.")

Detected 350 anomalies across all tickers.


In [13]:
clustering_df = pd.read_csv(proc_dir / "market_data_anomalies.csv")
clean_df = clustering_df.dropna(subset=features).copy()

scaler = StandardScaler()
X_scaled = scaler.fit_transform(clean_df[features])

optimal_k = 5
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
clean_df['Cluster'] = kmeans.fit_predict(X_scaled)

model_root = project_root / "Models"
model_root.mkdir(parents=True, exist_ok=True)
joblib.dump(kmeans, model_root / "kmeans_model.pkl")
joblib.dump(scaler, model_root / "scaler.pkl")
clean_df.to_csv(proc_dir / "market_data_clusters.csv", index=False)

def find_similar_tickers(query_features_dict, df, kmeans_model, scaler_model, top_n=5):
    query_df = pd.DataFrame([query_features_dict])
    query_scaled = scaler_model.transform(query_df[features])
    
    query_cluster = kmeans_model.predict(query_scaled)[0]
    
    latest_states = df.sort_values('Date').groupby('Ticker').last().reset_index()
    cluster_peers = latest_states[latest_states['Cluster'] == query_cluster].copy()
    
    if cluster_peers.empty:
        return pd.DataFrame()
        
    peer_features_scaled = scaler_model.transform(cluster_peers[features])
    distances = cdist(query_scaled, peer_features_scaled, metric='euclidean')[0]
    
    cluster_peers['Distance'] = distances
    return cluster_peers.nsmallest(top_n, 'Distance')[['Ticker', 'Cluster', 'Distance'] + features]

query_features = {
    'RSI_14': 65.5, 
    'SMA_Ratio_5_20': 1.02, 
    'MACD_Histogram': 0.5,
    'BB_Position': 0.7, 
    'Volume_Ratio': 1.3
}

similar = find_similar_tickers(query_features, clean_df, kmeans, scaler, top_n=3)
print(similar)

   Ticker  Cluster  Distance     RSI_14  SMA_Ratio_5_20  MACD_Histogram  \
29  MC.PA        2  0.548191  61.123779        1.011473        0.412827   
12    AZN        2  0.572248  56.227176        1.012099        0.041974   
40    SAP        2  0.830896  52.612893        1.010896        0.911332   

    BB_Position  Volume_Ratio  
29     0.750263      1.133350  
12     0.665719      1.248418  
40     0.775155      1.421570  


In [14]:
df = pd.read_csv(proc_dir / "market_data_clusters.csv")
df['Target'] = (df.groupby('Ticker')['Close'].shift(-1) > df['Close']).astype(int)
features = [
    'RSI_14', 
    'MACD_Histogram',
    'BB_Position', 
    'SMA_Ratio_5_20',
    'Volume_Ratio', 
    'Is_Anomaly', 
    'Cluster'
]
clean_df = df.dropna(subset=['Target'] + features).copy()
split_idx = int(len(clean_df) * 0.8)
train_df = clean_df.iloc[:split_idx]
test_df = clean_df.iloc[split_idx:]
X_train, y_train = train_df[features], train_df['Target']
X_test, y_test = test_df[features], test_df['Target']

xgb_model = xgb.XGBClassifier(n_estimators=500,learning_rate=0.05,max_depth=5,random_state=42,eval_metric='logloss')
xgb_model.fit(X_train, y_train,eval_set=[(X_test, y_test)],verbose=False)

y_pred = xgb_model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Directional Accuracy: {acc:.2%}")

joblib.dump(xgb_model, model_root / "trend_model.pkl")

Directional Accuracy: 56.64%


['d:\\VSCode\\Projects\\Stock-Market-Trend-Analysis\\Models\\trend_model.pkl']

In [16]:
# ============================================================================
# LSTM MODEL FOR MULTIPLE TICKERS - TIME SERIES PREDICTION
# ============================================================================
# Note: All required imports are in Cell 1

# Load the processed data
current_dir = Path.cwd()
project_root = current_dir if current_dir.name != 'Notebooks' else current_dir.parent
proc_dir = project_root / "Data"
model_root = project_root / "Models"

print("\n" + "="*70)
print("LSTM TIME SERIES PREDICTION FOR MULTIPLE TICKERS")
print("="*70)

# Load data
df = pd.read_csv(proc_dir / "market_data_clusters.csv")
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(['Ticker', 'Date']).reset_index(drop=True)

print(f"\n✓ Loaded data: {len(df)} rows, {df['Ticker'].nunique()} tickers")
print(f"✓ Date range: {df['Date'].min()} to {df['Date'].max()}")

# Feature selection for LSTM
feature_cols = [
    'Open', 'High', 'Low', 'Close', 'Volume',
    'RSI_14', 'SMA_Ratio_5_20', 'MACD_Histogram', 
    'BB_Position', 'Volume_Ratio'
]

# Function to create sequences for LSTM
def create_sequences(data, lookback=60, forecast_horizon=1):
    """
    Create sequences for LSTM training
    Args:
        data: numpy array of shape (n_samples, n_features)
        lookback: number of time steps to look back
        forecast_horizon: number of steps ahead to predict
    Returns:
        X: sequences of shape (n_sequences, lookback, n_features)
        y: targets of shape (n_sequences, forecast_horizon)
    """
    X, y = [], []
    for i in range(len(data) - lookback - forecast_horizon + 1):
        X.append(data[i:i+lookback])
        y.append(data[i+lookback:i+lookback+forecast_horizon, 3])  # Close price is at index 3
    return np.array(X), np.array(y)

# Prepare data for each ticker
LOOKBACK = 60  # Use 60 days to predict next day
FORECAST_HORIZON = 1  # Predict 1 day ahead

all_X_train, all_y_train = [], []
all_X_test, all_y_test = [], []
ticker_scalers = {}
ticker_models = {}

tickers = df['Ticker'].unique()
print(f"\n{'='*70}")
print(f"Processing {len(tickers)} tickers...")
print(f"{'='*70}")

# Process each ticker
for ticker in tickers:
    ticker_data = df[df['Ticker'] == ticker].copy()
    
    if len(ticker_data) < LOOKBACK + 20:  # Need minimum data
        print(f"⚠ Skipping {ticker}: Insufficient data ({len(ticker_data)} days)")
        continue
    
    # Extract features
    ticker_features = ticker_data[feature_cols].values
    
    # Scale the data (per ticker)
    scaler = MinMaxScaler(feature_range=(0, 1))
    ticker_scaled = scaler.fit_transform(ticker_features)
    ticker_scalers[ticker] = scaler
    
    # Create sequences
    X, y = create_sequences(ticker_scaled, LOOKBACK, FORECAST_HORIZON)
    
    if len(X) < 20:  # Need minimum sequences
        print(f"⚠ Skipping {ticker}: Insufficient sequences ({len(X)})")
        continue
    
    # Split train/test (80/20) - maintaining temporal order
    split_idx = int(len(X) * 0.8)
    X_train_ticker = X[:split_idx]
    y_train_ticker = y[:split_idx]
    X_test_ticker = X[split_idx:]
    y_test_ticker = y[split_idx:]
    
    # Build LSTM model for this ticker
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=(LOOKBACK, len(feature_cols))),
        Dropout(0.2),
        LSTM(50, return_sequences=True),
        Dropout(0.2),
        LSTM(50),
        Dropout(0.2),
        Dense(25),
        Dense(FORECAST_HORIZON)
    ])
    
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    
    # Callbacks
    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    # Train the model
    print(f"\n📊 Training LSTM for {ticker}...")
    print(f"   Training samples: {len(X_train_ticker)}, Test samples: {len(X_test_ticker)}")
    
    history = model.fit(
        X_train_ticker, y_train_ticker,
        validation_data=(X_test_ticker, y_test_ticker),
        epochs=50,
        batch_size=32,
        callbacks=[early_stop],
        verbose=0
    )
    
    # Evaluate
    train_pred = model.predict(X_train_ticker, verbose=0)
    test_pred = model.predict(X_test_ticker, verbose=0)
    
    # Inverse transform predictions (denormalize)
    # Create dummy arrays for inverse transform
    train_pred_full = np.zeros((len(train_pred), len(feature_cols)))
    train_pred_full[:, 3] = train_pred.flatten()  # Close price at index 3
    train_pred_denorm = scaler.inverse_transform(train_pred_full)[:, 3]
    
    test_pred_full = np.zeros((len(test_pred), len(feature_cols)))
    test_pred_full[:, 3] = test_pred.flatten()
    test_pred_denorm = scaler.inverse_transform(test_pred_full)[:, 3]
    
    # Denormalize actual values
    y_train_full = np.zeros((len(y_train_ticker), len(feature_cols)))
    y_train_full[:, 3] = y_train_ticker.flatten()
    y_train_denorm = scaler.inverse_transform(y_train_full)[:, 3]
    
    y_test_full = np.zeros((len(y_test_ticker), len(feature_cols)))
    y_test_full[:, 3] = y_test_ticker.flatten()
    y_test_denorm = scaler.inverse_transform(y_test_full)[:, 3]
    
    # Calculate metrics
    train_rmse = np.sqrt(mean_squared_error(y_train_denorm, train_pred_denorm))
    test_rmse = np.sqrt(mean_squared_error(y_test_denorm, test_pred_denorm))
    train_mae = mean_absolute_error(y_train_denorm, train_pred_denorm)
    test_mae = mean_absolute_error(y_test_denorm, test_pred_denorm)
    test_r2 = r2_score(y_test_denorm, test_pred_denorm)
    
    print(f"   ✓ Train RMSE: ${train_rmse:.2f}, MAE: ${train_mae:.2f}")
    print(f"   ✓ Test  RMSE: ${test_rmse:.2f}, MAE: ${test_mae:.2f}, R²: {test_r2:.4f}")
    
    # Store model and data
    ticker_models[ticker] = model
    all_X_train.append(X_train_ticker)
    all_y_train.append(y_train_denorm)
    all_X_test.append(X_test_ticker)
    all_y_test.append(y_test_denorm)

print(f"\n{'='*70}")
print(f"✓ Successfully trained LSTM models for {len(ticker_models)} tickers")
print(f"{'='*70}")

# Aggregate results across all tickers
if len(ticker_models) > 0:
    all_test_predictions = []
    all_test_actuals = []
    
    for ticker in ticker_models.keys():
        ticker_data = df[df['Ticker'] == ticker].copy()
        ticker_features = ticker_data[feature_cols].values
        scaler = ticker_scalers[ticker]
        ticker_scaled = scaler.transform(ticker_features)
        X, y = create_sequences(ticker_scaled, LOOKBACK, FORECAST_HORIZON)
        
        split_idx = int(len(X) * 0.8)
        X_test_ticker = X[split_idx:]
        y_test_ticker = y[split_idx:]
        
        model = ticker_models[ticker]
        test_pred = model.predict(X_test_ticker, verbose=0)
        
        # Denormalize
        test_pred_full = np.zeros((len(test_pred), len(feature_cols)))
        test_pred_full[:, 3] = test_pred.flatten()
        test_pred_denorm = scaler.inverse_transform(test_pred_full)[:, 3]
        
        y_test_full = np.zeros((len(y_test_ticker), len(feature_cols)))
        y_test_full[:, 3] = y_test_ticker.flatten()
        y_test_denorm = scaler.inverse_transform(y_test_full)[:, 3]
        
        all_test_predictions.extend(test_pred_denorm)
        all_test_actuals.extend(y_test_denorm)
    
    # Overall metrics
    overall_rmse = np.sqrt(mean_squared_error(all_test_actuals, all_test_predictions))
    overall_mae = mean_absolute_error(all_test_actuals, all_test_predictions)
    overall_r2 = r2_score(all_test_actuals, all_test_predictions)
    
    print(f"\n{'='*70}")
    print("OVERALL MODEL PERFORMANCE (All Tickers Combined)")
    print(f"{'='*70}")
    print(f"Test RMSE: ${overall_rmse:.2f}")
    print(f"Test MAE:  ${overall_mae:.2f}")
    print(f"Test R²:   {overall_r2:.4f}")
    print(f"{'='*70}")
    
    # Save models
    print(f"\n💾 Saving models...")
    for ticker, model in ticker_models.items():
        model_path = model_root / f"lstm_model_{ticker}.h5"
        model.save(model_path)
    
    # Save scalers
    scaler_path = model_root / "lstm_scalers.pkl"
    joblib.dump(ticker_scalers, scaler_path)
    
    print(f"✓ Models saved to {model_root}")
    
    # Visualization for a sample ticker
    sample_ticker = list(ticker_models.keys())[0]
    print(f"\n📈 Generating visualization for sample ticker: {sample_ticker}")
    
    ticker_data = df[df['Ticker'] == sample_ticker].copy()
    ticker_features = ticker_data[feature_cols].values
    scaler = ticker_scalers[sample_ticker]
    ticker_scaled = scaler.transform(ticker_features)
    X, y = create_sequences(ticker_scaled, LOOKBACK, FORECAST_HORIZON)
    
    split_idx = int(len(X) * 0.8)
    X_test_ticker = X[split_idx:]
    y_test_ticker = y[split_idx:]
    
    model = ticker_models[sample_ticker]
    test_pred = model.predict(X_test_ticker, verbose=0)
    
    # Denormalize
    test_pred_full = np.zeros((len(test_pred), len(feature_cols)))
    test_pred_full[:, 3] = test_pred.flatten()
    test_pred_denorm = scaler.inverse_transform(test_pred_full)[:, 3]
    
    y_test_full = np.zeros((len(y_test_ticker), len(feature_cols)))
    y_test_full[:, 3] = y_test_ticker.flatten()
    y_test_denorm = scaler.inverse_transform(y_test_full)[:, 3]
    
    # Plot
    plt.figure(figsize=(14, 6))
    plt.plot(y_test_denorm, label='Actual Close Price', alpha=0.7, linewidth=2)
    plt.plot(test_pred_denorm, label='Predicted Close Price', alpha=0.7, linewidth=2)
    plt.title(f'LSTM Predictions vs Actual - {sample_ticker}', fontsize=14, fontweight='bold')
    plt.xlabel('Time Steps', fontsize=12)
    plt.ylabel('Close Price ($)', fontsize=12)
    plt.legend(fontsize=11)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print(f"\n✅ LSTM implementation complete!")
else:
    print("\n⚠ No models were trained. Check data requirements.")


LSTM TIME SERIES PREDICTION FOR MULTIPLE TICKERS

✓ Loaded data: 7000 rows, 50 tickers
✓ Date range: 2025-08-04 00:00:00 to 2026-01-13 00:00:00

Processing 50 tickers...


TypeError: 'NoneType' object is not callable